In [1]:
import os


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.data import Data
import torch_geometric.nn as pyg_nn
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Subset
from collections import defaultdict

if torch.backends.mps.is_available():
    print("MPS backend is available!")
else:
    print("MPS backend is not available.")

device = torch.device("mps")

df = pd.read_parquet('stock_data/processed/merged_stock_data_with_enhanced_features.parquet')
df.head()

/opt/anaconda3/envs/Universal/lib/python3.9/site-packages/torch/utils/_pytree.py:183: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


MPS backend is available!


Automobile and Auto Components                      \
                                              APOLLOTYRE                       
                                                   price return_1d return_5d   
2022-01-10 00:00:00+05:30                     223.718369 -0.002796  0.047097   
2022-01-11 00:00:00+05:30                     223.092926 -0.002796  0.047097   
2022-01-12 00:00:00+05:30                     224.824921  0.007764  0.047097   
2022-01-13 00:00:00+05:30                     226.075821  0.005564  0.047097   
2022-01-14 00:00:00+05:30                     222.852371 -0.014258  0.047097   

                                                                         \
                                                                          
                          return_21d log_return      SMA_20      SMA_50   
2022-01-10 00:00:00+05:30   -0.02086  -0.002800  217.155955  199.445156   
2022-01-11 00:00:00+05:30   -0.02086  -0.002800  217.155955  199.445156   
2022-01-12 00:00:00+05:30   -0.02086   0.007734  217.155955  199.445156   
2022-01-13 00:00:00+05:30   -0.02086   0.005548  217.155955  199.445156   
2022-01-14 00:00:00+05:30   -0.02086  -0.014361  217.155955  199.445156   

                                                               ...  \
                                                               ...   
                               EMA_10      EMA_20      EMA_50  ...   
2022-01-10 00:00:00+05:30  223.718369  223.718369  223.718369  ...   
2022-01-11 00:00:00+05:30  223.604652  223.658803  223.693841  ...   
2022-01-12 00:00:00+05:30  223.826519  223.769861  223.738197  ...   
2022-01-13 00:00:00+05:30  224.235483  223.989477  223.829869  ...   
2022-01-14 00:00:00+05:30  223.984008  223.881181  223.791536  ...   

                                 Textiles                            \
                               WELSPUNLIV                             
                          dist_from_sma50 dist_from_ema20      MACD   
2022-01-10 00:00:00+05:30       -0.189072        0.000000  0.000000   
2022-01-11 00:00:00+05:30       -0.189072       -0.022249 -0.297982   
2022-01-12 00:00:00+05:30       -0.189072       -0.013623 -0.440638   
2022-01-13 00:00:00+05:30       -0.189072        0.009352 -0.257350   
2022-01-14 00:00:00+05:30       -0.189072       -0.007274 -0.321389   

                                                                           \
                                                                            
                          MACD_signal        RSI volatility_20d  BB_width   
2022-01-10 00:00:00+05:30    0.000000  36.533375       0.028095  0.221401   
2022-01-11 00:00:00+05:30   -0.059596  36.533375       0.028095  0.221401   
2022-01-12 00:00:00+05:30   -0.135805  36.533375       0.028095  0.221401   
2022-01-13 00:00:00+05:30   -0.160114  36.533375       0.028095  0.221401   
2022-01-14 00:00:00+05:30   -0.192369  36.533375       0.028095  0.221401   

                                                                      
                                                                      
                          ema_cross_signal is_month_end return_ratio  
2022-01-10 00:00:00+05:30                0            0    -0.024534  
2022-01-11 00:00:00+05:30                0            0    -0.024534  
2022-01-12 00:00:00+05:30                0            0     0.007378  
2022-01-13 00:00:00+05:30                0            0     0.024301  
2022-01-14 00:00:00+05:30                0            0    -0.017225  

[5 rows x 10584 columns]

In [3]:
len(df.columns.levels[1])

441

In [4]:

df.index = pd.to_datetime(df.index)  


# start_date = "2022-06-10"
start_date = "2022-06-01"
end_date = "2023-10-30"

df_train = df.loc[start_date:end_date]

end_val_date = "2024-03-31"

df_val_date  = df.loc[end_date:end_val_date]

print(df_train.columns.levels[2])
print(df_train.iloc[0].shape)
print(df_train.isna().sum())
df_train.head()
# print(df_val_date.head())

Index(['BB_width', 'EMA_10', 'EMA_20', 'EMA_50', 'MACD', 'MACD_signal', 'RSI',
       'SMA_20', 'SMA_50', 'WMA_10', 'WMA_20', 'WMA_50', 'dist_from_ema20',
       'dist_from_sma20', 'dist_from_sma50', 'ema_cross_signal',
       'is_month_end', 'log_return', 'price', 'return_1d', 'return_21d',
       'return_5d', 'return_ratio', 'volatility_20d'],
      dtype='object')
(10584,)
Automobile and Auto Components  APOLLOTYRE  price               0
                                            return_1d           0
                                            return_5d           0
                                            return_21d          0
                                            log_return          0
                                                               ..
Textiles                        WELSPUNLIV  volatility_20d      0
                                            BB_width            0
                                            ema_cross_signal    0
                           

Automobile and Auto Components                      \
                                              APOLLOTYRE                       
                                                   price return_1d return_5d   
2022-06-01 00:00:00+05:30                     211.546158  0.006870  0.033130   
2022-06-02 00:00:00+05:30                     213.518723  0.009325  0.039344   
2022-06-03 00:00:00+05:30                     210.054703 -0.016223  0.014641   
2022-06-06 00:00:00+05:30                     207.745346 -0.010994 -0.018636   
2022-06-07 00:00:00+05:30                     202.934189 -0.023159 -0.034120   

                                                                         \
                                                                          
                          return_21d log_return      SMA_20      SMA_50   
2022-06-01 00:00:00+05:30   0.085143   0.006846  201.416283  193.762229   
2022-06-02 00:00:00+05:30   0.109500   0.009281  202.484357  194.404036   
2022-06-03 00:00:00+05:30   0.093140  -0.016357  203.542810  194.947696   
2022-06-06 00:00:00+05:30   0.099847  -0.011055  204.637347  195.462489   
2022-06-07 00:00:00+05:30   0.091898  -0.023431  205.221900  195.929170   

                                                               ...  \
                                                               ...   
                               EMA_10      EMA_20      EMA_50  ...   
2022-06-01 00:00:00+05:30  207.586050  203.544625  198.161059  ...   
2022-06-02 00:00:00+05:30  208.664718  204.494539  198.763320  ...   
2022-06-03 00:00:00+05:30  208.917442  205.024078  199.206120  ...   
2022-06-06 00:00:00+05:30  208.704334  205.283247  199.540991  ...   
2022-06-07 00:00:00+05:30  207.655217  205.059527  199.674058  ...   

                                 Textiles                            \
                               WELSPUNLIV                             
                          dist_from_sma50 dist_from_ema20      MACD   
2022-06-01 00:00:00+05:30       -0.194903       -0.054385 -5.196286   
2022-06-02 00:00:00+05:30       -0.151508       -0.009689 -4.645313   
2022-06-03 00:00:00+05:30       -0.122049        0.016096 -4.005750   
2022-06-06 00:00:00+05:30       -0.113223        0.017715 -3.439152   
2022-06-07 00:00:00+05:30       -0.134138       -0.011330 -3.126888   

                                                                           \
                                                                            
                          MACD_signal        RSI volatility_20d  BB_width   
2022-06-01 00:00:00+05:30   -5.903133  51.452274       0.059091  0.314735   
2022-06-02 00:00:00+05:30   -5.651569  53.693924       0.060170  0.279709   
2022-06-03 00:00:00+05:30   -5.322405  59.785546       0.060612  0.242541   
2022-06-06 00:00:00+05:30   -4.945755  56.395365       0.060535  0.209062   
2022-06-07 00:00:00+05:30   -4.581981  52.819812       0.060777  0.171931   

                                                                      
                                                                      
                          ema_cross_signal is_month_end return_ratio  
2022-06-01 00:00:00+05:30                0            0     0.026779  
2022-06-02 00:00:00+05:30                0            0     0.046200  
2022-06-03 00:00:00+05:30                0            0     0.027778  
2022-06-06 00:00:00+05:30                0            0     0.003465  
2022-06-07 00:00:00+05:30                0            0    -0.029696  

[5 rows x 10584 columns]

In [5]:
class HierarchicalStockDataset(Dataset):
    def __init__(self, multiindex_df, sequence_length=5):
        self.sequence_length = sequence_length
        self.samples = []

        # Create mappings with persistence support
        self.industry_map = {industry: idx for idx, industry in enumerate(multiindex_df.columns.levels[0])}
        self.company_map = {company: idx for idx, company in enumerate(multiindex_df.columns.levels[1])}

        for industry_id, industry in enumerate(multiindex_df.columns.levels[0]):
            for company_id, company in enumerate(multiindex_df.columns.levels[1]):
                if industry not in self.industry_map:
                    self.industry_map[industry] = industry_id
                if company not in self.company_map:
                    self.company_map[company] = company_id
                
                try:
                    # Use xs to select from multi-level columns properly
                    company_df = multiindex_df.xs((industry, company), axis=1, level=[0,1]).copy()

                    if 'return_ratio' not in company_df.columns:
                        print(f"Warning: 'return_ratio' not found for {industry}, {company}")
                        # print(company_df.head())
                        # print(f"Warning: 'return_ratio' not found for {industry}, {company}")
                        continue
                        
                    features = company_df.drop('return_ratio', axis=1).values
                    labels = company_df['return_ratio'].values
                    
                    # Create sequences with proper future prediction
                    for i in range(len(features) - self.sequence_length-1):
                        self.samples.append({
                            'features': features[i:i+self.sequence_length],
                            'industry': self.industry_map[industry],
                            'company': self.company_map[company],
                            'return_ratio': labels[i + self.sequence_length+1],
                            'movements': (labels[i + self.sequence_length + 1]>0),
                        })
                except KeyError:
                    print(f"Could not find data for industry={industry}, company={company}")
                    continue

    def __getitem__(self, idx):
        sample = self.samples[idx]
        return (
            torch.tensor(sample['features'], dtype=torch.float32),  # (seq_len, n_features)
            torch.tensor(sample['industry'], dtype=torch.long),
            torch.tensor(sample['company'], dtype=torch.long),
            torch.tensor(sample['return_ratio'], dtype=torch.float32),
            torch.tensor(sample['movements'], dtype=torch.float32)  
        )
        
    def __len__(self):
        return len(self.samples)


In [6]:
class TransformerSequentialLearner(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2, nhead=4, dropout=0.1):
        super().__init__()
        
        # Positional encoding is important for transformers to understand sequence order
        self.pos_encoder = nn.Parameter(torch.zeros(1, 100, hidden_size))  # Max 100 time steps
        
        # Project input to hidden dimension if needed
        self.input_proj = nn.Linear(input_size, hidden_size) if input_size != hidden_size else nn.Identity()
        
        # Transformer encoder
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=hidden_size, 
            nhead=nhead, 
            dim_feedforward=hidden_size*4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)
        
        # Output projection (optional)
        self.output_proj = nn.Linear(hidden_size, hidden_size)
        
    def forward(self, x):
        # x shape: [batch_size, seq_len, features]
        seq_len = x.size(1)
        
        # Project input if needed
        x = self.input_proj(x)
        
        # Add positional encoding
        x = x + self.pos_encoder[:, :seq_len, :]
        
        # Pass through transformer encoder
        transformer_output = self.transformer_encoder(x)
        
        # Two outputs to match original API:
        # 1. Context vector (average pooling across sequence)
        context = torch.mean(transformer_output, dim=1)
        
        return context, transformer_output

In [7]:

train_dataset = HierarchicalStockDataset(df_train)
company_id_to_name = {idx: name for name, idx in train_dataset.company_map.items()}
# val_dataset = HierarchicalStockDataset(df_val_date)


In [8]:
seq_len = train_dataset.__len__()
seq_len



152145

In [9]:
# train_loader = DataLoader(train_dataset, batch_size=32)
# val_loader = DataLoader(val_dataset, batch_size=32)

In [10]:
# i = 0
# for features, industry, company, returns, movements in train_loader:
#     if company[0] ==  29:
#         print(f"Batch {i}:")
#         print(f"Features: {features.shape}")
#         print(f"Industry: {industry.shape}")
#         print(f"Company: {company.shape}")
#         print(f"returns: {returns.shape}")
#         print(f"movements: {movements.shape}")

#         i += 1
#     else:
#         print(f"Batch {i}:")
#         break
# # Hyperparameters
     
    

In [12]:
HIDDEN_SIZE = 16  # As specified in Section 4.1
DEVICE = torch.device("mps")  # Use MPS device if available

# Initialize model WITH PAPER-SPECIFIED DIMENSIONS
model = TransformerSequentialLearner(
    input_size=train_dataset[0][0].shape[1],  # num_features
    hidden_size=HIDDEN_SIZE
).to(DEVICE)

# Create dictionaries to store embeddings and metadata
attentive_embeddings = {}  # Will store weekly contexts for each company
sector_stock_map = defaultdict(set)
stock_sector_map = defaultdict(set)
stock_returns_map = defaultdict(list)
stock_movements_map = defaultdict(list)

# Group sequences by company for orderly processing
company_sequences = defaultdict(list)

# First pass: organize all samples by company for sequential processing
for idx in range(len(train_dataset)):
    features, industry_id, company_id, return_ratio, movements = train_dataset[idx]
    company_id = company_id.item()
    industry_id = industry_id.item()
    
    company_sequences[company_id].append({
        'features': features,
        'industry_id': industry_id,
        'return_ratio': return_ratio.item(),
        'movements': movements.item(),
        'idx': idx  # Keep track of original index if needed
    })
    
    # Map companies to sectors
    sector_stock_map[industry_id].add(company_id)
    stock_sector_map[company_id].add(industry_id)

# Second pass: process each company's data in sequence to preserve temporal order
model.eval()
with torch.no_grad():
    for company_id, sequences in company_sequences.items():
        # Sort by original index to maintain temporal ordering
        sequences.sort(key=lambda x: x['idx'])
        
        # Process all sequences for this company in order
        company_attentive_embeddings = []
        
        for seq in sequences:
            features = seq['features'].unsqueeze(0).to(DEVICE)  # Add batch dimension
            industry_id = seq['industry_id']
            return_ratio = seq['return_ratio']
            movements = seq['movements']
            
            # Store return and movement information
            stock_returns_map[company_id].append(return_ratio)
            stock_movements_map[company_id].append(movements)
            
            # Get embeddings
            context, _ = model(features)
            company_attentive_embeddings.append(context.squeeze(0).cpu().numpy())
        
        # Store the entire temporal sequence for this company
        attentive_embeddings[company_id] = np.stack(company_attentive_embeddings, axis=0)

print(f"Attentive embeddings processed: {len(attentive_embeddings)} companies with properly ordered sequences")

Attentive embeddings processed: 441 companies with properly ordered sequences


In [13]:
print(len(attentive_embeddings))
attentive_embeddings[2].shape  # Example for company 0

441


(345, 16)

In [14]:
for key in sector_stock_map:
    sector_stock_map[key] = list(set(sector_stock_map[key]))
print(f"Sector stock map: {sector_stock_map}")


Sector stock map: defaultdict(<class 'set'>, {0: [131, 69, 136, 392, 270, 400, 82, 279, 281, 411, 284, 29, 348, 31, 221, 417, 95, 226, 421, 104, 360, 43, 48, 179, 308, 371, 438, 125, 378, 61], 1: [128, 129, 3, 135, 9, 266, 139, 141, 15, 399, 273, 401, 402, 408, 25, 26, 32, 160, 35, 163, 165, 419, 167, 171, 430, 178, 57, 58, 59, 187, 63, 326, 330, 331, 78, 208, 338, 88, 346, 222, 99, 361, 236, 109, 365, 366, 368, 113, 243, 245, 246, 251, 381, 383], 2: [1, 257, 387, 137, 140, 142, 18, 155, 418, 291, 38, 168, 426, 47, 53, 55, 313, 188, 320, 321, 341, 90, 91, 97, 103, 233, 370, 115, 116, 374, 376, 126], 3: [384, 259, 132, 292, 166, 293, 324, 397, 242, 213, 214, 247, 347, 351], 4: [224, 65, 225, 164, 8, 201, 363, 303, 112, 337, 21, 415], 5: [68, 336, 275, 20, 403, 84, 85, 33, 424, 427, 428, 173, 107, 238, 239, 433, 240, 54, 119, 60], 6: [66, 255, 6, 200, 202, 406, 215, 89, 350, 290, 355, 232, 235, 304, 432, 117, 439, 121, 123, 127], 7: [130, 389, 71, 269, 334, 80, 145, 149, 409, 56, 345, 15

In [15]:
# Add sector_id attribute to each graph when creating them
weekly_intra_sector_graphs = defaultdict(lambda: defaultdict(list))

# Process each timepoint (week) separately
for week_idx in range(attentive_embeddings[0].shape[0]):
    print(f"Processing week {week_idx}")
    
    # Group by sector
    for sector_id, sector_stocks in sector_stock_map.items():
        sector_features = []
        valid_indices = []
        
        # Collect stock embeddings in this sector-week
        for stock_idx in sector_stocks:
            if stock_idx in attentive_embeddings and week_idx < len(attentive_embeddings[stock_idx]):
                # Convert NumPy array to PyTorch tensor
                tensor_embedding = torch.tensor(attentive_embeddings[stock_idx][week_idx], dtype=torch.float32)
                sector_features.append(tensor_embedding)
                valid_indices.append(stock_idx)
        
        # Create graph if >= 2 stocks
        if len(sector_features) >= 2:
            edge_index = []
            num_nodes = len(valid_indices)
            
            # Fully-connected edges without self-loops
            for i in range(num_nodes):
                for j in range(num_nodes):
                    if i != j:
                        edge_index.append([i, j])
            
            # Store the graph with additional sector_id attribute
            weekly_intra_sector_graphs[week_idx][sector_id] = Data(
                x=torch.stack(sector_features),
                edge_index=torch.tensor(edge_index).t().contiguous(),
                original_indices=valid_indices,
                sector_id=sector_id  # Store the sector_id as an attribute
            )

Processing week 0
Processing week 1
Processing week 2
Processing week 3
Processing week 4
Processing week 5
Processing week 6
Processing week 7
Processing week 8
Processing week 9
Processing week 10
Processing week 11
Processing week 12
Processing week 13
Processing week 14
Processing week 15
Processing week 16
Processing week 17
Processing week 18
Processing week 19
Processing week 20
Processing week 21
Processing week 22
Processing week 23
Processing week 24
Processing week 25
Processing week 26
Processing week 27
Processing week 28
Processing week 29
Processing week 30
Processing week 31
Processing week 32
Processing week 33
Processing week 34
Processing week 35
Processing week 36
Processing week 37
Processing week 38
Processing week 39
Processing week 40
Processing week 41
Processing week 42
Processing week 43
Processing week 44
Processing week 45
Processing week 46
Processing week 47
Processing week 48
Processing week 49
Processing week 50
Processing week 51
Processing week 52
Pro

In [16]:
print(len(weekly_intra_sector_graphs))
print(len(weekly_intra_sector_graphs[0]))
print(weekly_intra_sector_graphs[0].keys())

345
19
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])


In [17]:
from torch_geometric.nn import GATConv
import torch.nn.functional as F

class IntraSectorGAT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Don't pre-initialize the model - we'll store models for different input sizes
        self.model_registry = {}
        self.out_dim = HIDDEN_SIZE  # Output dimension stays fixed at 16
    
    def forward(self, data):
        # Extract node features and edge structure
        x, edge_index = data.x, data.edge_index
        
        # Ensure x is 2D tensor
        if x.dim() != 2:
            x = x.reshape(-1, x.size(-1))
        
        # Get input dimension for this batch
        in_channels = x.size(1)
        
        # Create or retrieve the appropriate model for this input dimension
        if in_channels not in self.model_registry:
            print(f"Creating new GATConv model for input dimension {in_channels}")
            conv = GATConv(
                in_channels=in_channels,
                out_channels=self.out_dim // 4,
                heads=4,
                concat=True,
                negative_slope=0.2,
                dropout=0.1
            ).to(x.device)
            self.model_registry[in_channels] = conv
        
        # Use the appropriate model
        conv = self.model_registry[in_channels]
        x = conv(x, edge_index)
        
        return x


In [18]:
model = IntraSectorGAT().to(device)
sector_embeddings = []

model.eval()
with torch.no_grad():
    for week_idx in weekly_intra_sector_graphs:
        for sector_id, graph in weekly_intra_sector_graphs[week_idx].items():
            # Ensure graph is on the correct device
            graph = graph.to(device)
            
            # Debug information
            print(f"Graph x shape: {graph.x.shape}, edge_index shape: {graph.edge_index.shape}")
            
            # Process the graph with our updated model
            out = model(graph)
            print(f"Output shape: {out.shape}")
            sector_embeddings.append({
                'embeddings': out.cpu(),
                'original_indices': graph.original_indices,
                'sector_id': graph.sector_id,
                'week_idx': week_idx
            })

Graph x shape: torch.Size([30, 16]), edge_index shape: torch.Size([2, 870])
Creating new GATConv model for input dimension 16
Output shape: torch.Size([30, 16])
Graph x shape: torch.Size([54, 16]), edge_index shape: torch.Size([2, 2862])
Output shape: torch.Size([54, 16])
Graph x shape: torch.Size([32, 16]), edge_index shape: torch.Size([2, 992])
Output shape: torch.Size([32, 16])
Graph x shape: torch.Size([14, 16]), edge_index shape: torch.Size([2, 182])
Output shape: torch.Size([14, 16])
Graph x shape: torch.Size([12, 16]), edge_index shape: torch.Size([2, 132])
Output shape: torch.Size([12, 16])
Graph x shape: torch.Size([20, 16]), edge_index shape: torch.Size([2, 380])
Output shape: torch.Size([20, 16])
Graph x shape: torch.Size([20, 16]), edge_index shape: torch.Size([2, 380])
Output shape: torch.Size([20, 16])
Graph x shape: torch.Size([26, 16]), edge_index shape: torch.Size([2, 650])
Output shape: torch.Size([26, 16])
Graph x shape: torch.Size([83, 16]), edge_index shape: torch.

In [19]:
print(len(sector_embeddings))
print(len(sector_embeddings[5]['embeddings']))


6555
20


In [20]:
# # Determine the total number of companies and embedding dimensions
# num_companies = len(attentive_embeddings)
# embedding_dim = next(iter(attentive_embeddings.values())).shape[1]  # Assuming all embeddings have the same dimension

# # Initialize the final_embeddings tensor
# final_embeddings = torch.zeros(num_companies, embedding_dim)

# # Populate the final_embeddings tensor
# for sector_data in sector_embeddings:
#     indices = sector_data['original_indices']
#     embeddings = sector_data['embeddings']
#     for i, idx in enumerate(indices):
#         final_embeddings[idx] = embeddings[i]

# print(f"Final sector-aware embeddings shape: {final_embeddings.shape}")  # Should match the number of companies and embedding dimensions # Should match attentive_embeddings

In [21]:
# final_embeddings

In [22]:
class AttentiveGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=False
        )
        self.attention = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False)
        )

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        gru_out, _ = self.gru(x)  # (batch_size, seq_len, hidden_size)
        
        # Compute attention weights
        attn_weights = F.softmax(
            self.attention(gru_out).squeeze(-1), 
            dim=-1
        )  # (batch_size, seq_len)
        
        # Apply attention
        context = torch.bmm(
            attn_weights.unsqueeze(1),  # (batch_size, 1, seq_len)
            gru_out                     # (batch_size, seq_len, hidden_size)
        ).squeeze(1)  # (batch_size, hidden_size)
        
        return context

class LongTermTransformerLearner(nn.Module):
    def __init__(self, input_size, hidden_size, lookback_weeks=4, num_layers=2, nhead=4):
        super().__init__()
        self.lookback = lookback_weeks
        self.transformer = TransformerSequentialLearner(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            nhead=nhead
        )
        
    def forward(self, sequential_embeddings):
        # Stack embeddings from lookback weeks
        seq_window = torch.stack(sequential_embeddings[-self.lookback:], dim=1)
        
        # Process through transformer
        context, _ = self.transformer(seq_window)
        return context


In [23]:
# Create dictionaries to store long-term embeddings by week then company
weekly_long_term_embeddings = defaultdict(lambda: defaultdict(dict))
SEQ_LENGTH = 4  # Number of weeks to look back (sliding window)

# Initialize long term sequential learners
long_term_g = LongTermTransformerLearner(
    input_size=sector_embeddings[0]['embeddings'].shape[1],
    hidden_size=HIDDEN_SIZE,
    lookback_weeks=SEQ_LENGTH
).to(device)

long_term_a = LongTermTransformerLearner(
    input_size=attentive_embeddings[0].shape[1],
    hidden_size=HIDDEN_SIZE,
    lookback_weeks=SEQ_LENGTH
).to(device)

# Process embeddings for each company on a weekly basis
with torch.no_grad():
    # First, organize the sector embeddings by company and week
    company_sector_embeddings = defaultdict(lambda: defaultdict(list))
    
    for sector_data in sector_embeddings:
        week_idx = sector_data['week_idx']
        original_indices = sector_data['original_indices']
        embeddings = sector_data['embeddings']
        
        # Map embeddings to each company in this sector for this week
        for i, company_idx in enumerate(original_indices):
            company_sector_embeddings[company_idx][week_idx] = embeddings[i]
    
    # Get the maximum week index
    max_week = max([max(weekly_data.keys()) for weekly_data in company_sector_embeddings.values()])
    
    # Process each week from SEQ_LENGTH onwards (need history)
    for current_week in range(SEQ_LENGTH, max_week + 1):
        print(f"Processing long-term embeddings for week {current_week}")
        
        # Process each company for this week
        for company_idx, weekly_data in company_sector_embeddings.items():
            # Check if we have data for this company in this week
            if current_week not in weekly_data:
                continue
                
            # Check if we have enough history for this company
            history_weeks = [w for w in range(current_week - SEQ_LENGTH, current_week)]
            if not all(w in weekly_data for w in history_weeks):
                continue
                
            # We have enough history, gather the sliding window of embeddings
            graph_window = []
            for w in history_weeks:
                if w in weekly_data:
                    graph_emb = torch.tensor(weekly_data[w]).unsqueeze(0).to(device)
                    graph_window.append(graph_emb)
            
            # Process attentive embeddings if available
            if company_idx in attentive_embeddings:
                attentive_seqs = attentive_embeddings[company_idx]
                
                # Find the indices in attentive_seqs that correspond to our history weeks
                # This requires mapping from week index to position in the sequence
                if len(attentive_seqs) >= SEQ_LENGTH:
                    # Get the most recent SEQ_LENGTH embeddings
                    attentive_window = []
                    for i in range(SEQ_LENGTH):
                        # Map to correct position - we want the last SEQ_LENGTH entries
                        seq_idx = len(attentive_seqs) - SEQ_LENGTH + i
                        if seq_idx >= 0 and seq_idx < len(attentive_seqs):
                            att_emb = torch.tensor(attentive_seqs[seq_idx], dtype=torch.float32).unsqueeze(0).to(device)
                            attentive_window.append(att_emb)
                    
                    # Only proceed if we have enough embeddings in both windows
                    if len(graph_window) == SEQ_LENGTH and len(attentive_window) == SEQ_LENGTH:
                        # Process through the long term learners
                        tau_G = long_term_g(graph_window)
                        tau_A = long_term_a(attentive_window)
                        
                        # Store the results organized by week then company
                        weekly_long_term_embeddings[current_week][company_idx] = {
                            'graph': tau_G.cpu().numpy(),
                            'attentive': tau_A.cpu().numpy()
                        }

    # Also store by company for backward compatibility
    long_term_graph_embeddings = {}
    long_term_attentive_embeddings = {}
    
    # Take the most recent week's embeddings for each company
    for week in sorted(weekly_long_term_embeddings.keys(), reverse=True):
        for company_idx, embeddings in weekly_long_term_embeddings[week].items():
            if company_idx not in long_term_graph_embeddings:
                long_term_graph_embeddings[company_idx] = embeddings['graph']
                long_term_attentive_embeddings[company_idx] = embeddings['attentive']

print(f"Generated weekly long-term embeddings for {len(weekly_long_term_embeddings)} weeks")
print(f"Generated company long-term embeddings for {len(long_term_graph_embeddings)} companies")

# Example of accessing embeddings:
print("\nExample of weekly embedding structure:")
sample_week = list(weekly_long_term_embeddings.keys())[0]
sample_company = list(weekly_long_term_embeddings[sample_week].keys())[0]
print(f"Week {sample_week}, Company {sample_company}:")
print(f"Graph embedding shape: {weekly_long_term_embeddings[sample_week][sample_company]['graph'].shape}")
print(f"Attentive embedding shape: {weekly_long_term_embeddings[sample_week][sample_company]['attentive'].shape}")

Processing long-term embeddings for week 4


/var/folders/gn/mtbdxhjn1697bp_qhyd6y0cr0000gn/T/ipykernel_75395/1424686975.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  graph_emb = torch.tensor(weekly_data[w]).unsqueeze(0).to(device)


Processing long-term embeddings for week 5
Processing long-term embeddings for week 6
Processing long-term embeddings for week 7
Processing long-term embeddings for week 8
Processing long-term embeddings for week 9
Processing long-term embeddings for week 10
Processing long-term embeddings for week 11
Processing long-term embeddings for week 12
Processing long-term embeddings for week 13
Processing long-term embeddings for week 14
Processing long-term embeddings for week 15
Processing long-term embeddings for week 16
Processing long-term embeddings for week 17
Processing long-term embeddings for week 18
Processing long-term embeddings for week 19
Processing long-term embeddings for week 20
Processing long-term embeddings for week 21
Processing long-term embeddings for week 22
Processing long-term embeddings for week 23
Processing long-term embeddings for week 24
Processing long-term embeddings for week 25
Processing long-term embeddings for week 26
Processing long-term embeddings for w

In [24]:
# Create a nested dictionary to store pooled embeddings by week and sector
weekly_sector_pooled_embeddings = defaultdict(dict)
embedding_dim = sector_embeddings[0]['embeddings'].shape[1]

print("Performing max pooling for each sector and week...")

# For each week
for week_idx in sorted(weekly_intra_sector_graphs.keys()):
    # For each sector in this week
    for sector_id in weekly_intra_sector_graphs[week_idx]:
        graph = weekly_intra_sector_graphs[week_idx][sector_id]
        
        # Apply max pooling across all stocks in this sector (Equation 11)
        # graph.x shape: [num_stocks_in_sector, embedding_dim]
        pooled_embedding = torch.max(graph.x, dim=0)[0]  # shape: [embedding_dim]
        
        # Store the pooled embedding
        weekly_sector_pooled_embeddings[week_idx][sector_id] = pooled_embedding

# Print statistics
total_pooled_embeddings = sum(len(sectors) for sectors in weekly_sector_pooled_embeddings.values())
print(f"Created pooled embeddings for {len(weekly_sector_pooled_embeddings)} weeks and {total_pooled_embeddings} sector-week combinations")

# Example: Display the pooled embedding for the first week and first sector
first_week = min(weekly_sector_pooled_embeddings.keys())
first_sector = min(weekly_sector_pooled_embeddings[first_week].keys())
print(f"\nExample pooled embedding shape for week {first_week}, sector {first_sector}: {weekly_sector_pooled_embeddings[first_week][first_sector].shape}")

Performing max pooling for each sector and week...
Created pooled embeddings for 345 weeks and 6555 sector-week combinations

Example pooled embedding shape for week 0, sector 0: torch.Size([16])


In [25]:
# Step 1: Intra-sector Graph Pooling
# (Your existing code remains unchanged)

# Step 2: Inter-sector Relation Modeling
class InterSectorGAT(torch.nn.Module):
    def __init__(self, in_channels=16, hidden_channels=16):
        super().__init__()
        # Following paper specifications
        self.conv1 = GATConv(
            in_channels=in_channels,
            out_channels=hidden_channels // 4,
            heads=4,
            concat=True
        )
        self.conv2 = GATConv(
            in_channels=hidden_channels,
            out_channels=hidden_channels,
            heads=1,
            concat=False
        )
    
    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x



In [26]:
# Step 2: Weekly Inter-sector Relation Modeling
weekly_inter_sector_embeddings = defaultdict(dict)

# Initialize the inter-sector GAT model
inter_sector_model = InterSectorGAT(
    in_channels=embedding_dim,  # Use dimension from pooled embeddings
    hidden_channels=HIDDEN_SIZE
).to(device)

inter_sector_model.eval()

print("Performing inter-sector relation modeling for each week...")

# Process each week separately
with torch.no_grad():
    for week_idx in sorted(weekly_sector_pooled_embeddings.keys()):
        # Get all sectors for this week
        sectors_this_week = weekly_sector_pooled_embeddings[week_idx]
        
        # Skip weeks with less than 2 sectors (need at least 2 for a graph)
        num_sectors = len(sectors_this_week)
        if num_sectors < 2:
            print(f"Week {week_idx}: Skipping - only {num_sectors} sectors found")
            continue
            
        # Create a mapping from sector IDs to indices
        sector_ids = list(sectors_this_week.keys())
        sector_id_to_idx = {sector_id: idx for idx, sector_id in enumerate(sector_ids)}
        
        # Create features tensor for all sectors in this week
        sector_features = torch.zeros((num_sectors, embedding_dim), dtype=torch.float32)
        
        # Populate the features tensor
        for sector_id, idx in sector_id_to_idx.items():
            sector_features[idx] = sectors_this_week[sector_id]
        
        # Create fully connected graph (no self-loops) as specified in the paper
        edge_indices = []
        for i in range(num_sectors):
            for j in range(num_sectors):
                if i != j:  # No self-loops
                    edge_indices.append([i, j])
        
        edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
        
        # Process through the GAT model
        sector_features = sector_features.to(device)
        edge_index = edge_index.to(device)
        inter_sector_embeddings = inter_sector_model(sector_features, edge_index)
        
        # Convert back to CPU and store by original sector ID
        inter_sector_embeddings = inter_sector_embeddings.cpu()
        
        # Store in the weekly dictionary
        for sector_id, idx in sector_id_to_idx.items():
            weekly_inter_sector_embeddings[week_idx][sector_id] = inter_sector_embeddings[idx]
        
        print(f"Week {week_idx}: Processed {num_sectors} sectors with fully-connected graph")

# Print statistics
total_processed = sum(len(sectors) for sectors in weekly_inter_sector_embeddings.values())
print(f"Generated inter-sector embeddings for {len(weekly_inter_sector_embeddings)} weeks and {total_processed} sector-week combinations")

# Example: Display the inter-sector embedding for the first week and first sector
if weekly_inter_sector_embeddings:
    first_week = min(weekly_inter_sector_embeddings.keys())
    first_sector = min(weekly_inter_sector_embeddings[first_week].keys())
    shape = weekly_inter_sector_embeddings[first_week][first_sector].shape
    print(f"\nExample inter-sector embedding shape for week {first_week}, sector {first_sector}: {shape}")

Performing inter-sector relation modeling for each week...
Week 0: Processed 19 sectors with fully-connected graph
Week 1: Processed 19 sectors with fully-connected graph
Week 2: Processed 19 sectors with fully-connected graph
Week 3: Processed 19 sectors with fully-connected graph
Week 4: Processed 19 sectors with fully-connected graph
Week 5: Processed 19 sectors with fully-connected graph
Week 6: Processed 19 sectors with fully-connected graph
Week 7: Processed 19 sectors with fully-connected graph
Week 8: Processed 19 sectors with fully-connected graph
Week 9: Processed 19 sectors with fully-connected graph
Week 10: Processed 19 sectors with fully-connected graph
Week 11: Processed 19 sectors with fully-connected graph
Week 12: Processed 19 sectors with fully-connected graph
Week 13: Processed 19 sectors with fully-connected graph
Week 14: Processed 19 sectors with fully-connected graph
Week 15: Processed 19 sectors with fully-connected graph
Week 16: Processed 19 sectors with full

In [27]:
class EmbeddingFusion(nn.Module):
    def __init__(self, attentive_dim, graph_dim, sector_dim, output_dim=16):
        super().__init__()
        self.input_dim = attentive_dim + graph_dim + sector_dim
        self.fusion_layer = nn.Linear(self.input_dim, output_dim)
        
    def forward(self, attentive_emb, graph_emb, sector_emb):
        """
        Fuses three embedding types into a single representation
        Args:
            attentive_emb: τA_i(sq) - Short-term attentive embeddings
            graph_emb: τG_i(sq) - Intra-sector graph embeddings
            sector_emb: τi(πc) - Inter-sector embeddings
        Returns:
            Fused embedding vector τF_i(sq)
        """
        # Remove the extra dimension from graph and attentive embeddings
        if graph_emb.dim() > 2:
            graph_emb = graph_emb.squeeze(1)
        if attentive_emb.dim() > 2:
            attentive_emb = attentive_emb.squeeze(1)
        
        # Now all embeddings should have shape [batch_size, features]
        concatenated = torch.cat([graph_emb, attentive_emb, sector_emb], dim=-1)
        
        # Apply fusion transformation with ReLU (Equation 13)
        fused = F.relu(self.fusion_layer(concatenated))
        
        return fused

In [28]:
class FinGAT(nn.Module):
    def __init__(self, attentive_dim=16, graph_dim=16, sector_dim=16, hidden_dim=64):
        super().__init__()
        
        # Embedding fusion layer
        self.fusion = EmbeddingFusion(
            attentive_dim=attentive_dim,
            graph_dim=graph_dim,
            sector_dim=sector_dim,
            output_dim=hidden_dim
        )
        
        # Task-specific layers for return ratio prediction and movement prediction
        # As per Equation 14
        self.return_ratio_layer = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Linear(hidden_dim//2, 1)
        )

        self.movement_layer = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Linear(hidden_dim//2, 1)
        )
        
    def forward(self, attentive_emb, graph_emb, sector_emb):
        """
        Forward pass through the FinGAT model
        Args:
            attentive_emb: Short-term attentive embeddings
            graph_emb: Intra-sector graph embeddings
            sector_emb: Inter-sector embeddings
        Returns:
            return_ratio_pred: Predicted return ratio
            movement_pred: Predicted movement probability (up/down)
        """
        # Fusion of embeddings
        fused_embedding = self.fusion(attentive_emb, graph_emb, sector_emb)
        
        # Task-specific predictions (Equation 14)
        return_ratio_pred = self.return_ratio_layer(fused_embedding).squeeze(-1)
        movement_pred = torch.sigmoid(self.movement_layer(fused_embedding)).squeeze(-1)
        
        return return_ratio_pred, movement_pred

In [29]:
class MultiTaskLoss:
    def __init__(self, alpha=0.5):
        """
        Multi-task loss function combining ranking loss and binary classification loss
        Args:
            alpha: Weight balancing factor between the two losses
        """
        self.alpha = alpha  # Balance between ranking loss and classification loss
        self.bce_loss = nn.BCELoss()
        
    def pairwise_ranking_loss(self, predictions, targets):
        """
        Pairwise ranking loss that encourages correct ordering of pairs
        Args:
            predictions: Predicted return ratios
            targets: Ground truth return ratios
        """
        # Create all possible pairs in the batch
        n = predictions.size(0)
        pairs_i, pairs_j = torch.triu_indices(n, n, offset=1)
        
        # Calculate differences in predictions and targets for all pairs
        pred_diff = predictions[pairs_i] - predictions[pairs_j]
        target_diff = targets[pairs_i] - targets[pairs_j]
        
        # Convert target differences to binary indicators of correct ordering
        target_sign = torch.sign(target_diff)
        
        # Ranking loss: encourage predicted differences to have same sign as target differences
        # max(0, -target_sign * pred_diff + margin)
        margin = 0.1
        loss = F.relu(-target_sign * pred_diff + margin)
        
        return loss.mean()
    
    def __call__(self, return_preds, return_targets, move_preds, move_targets):
        """
        Calculate combined multi-task loss
        Args:
            return_preds: Predicted return ratios
            return_targets: Actual return ratios
            move_preds: Predicted movement probabilities
            move_targets: Actual movement labels (0/1)
        """
        ranking_loss = self.pairwise_ranking_loss(return_preds, return_targets)
        movement_loss = self.bce_loss(move_preds, move_targets)
        
        # Combine the losses using alpha as weight
        combined_loss = self.alpha * ranking_loss + (1 - self.alpha) * movement_loss
        
        return combined_loss, ranking_loss, movement_loss

In [30]:
# class MultiTaskLoss:
#     def __init__(self, alpha=0.8):
#         """
#         Multi-task loss function combining MSE loss for return ratio and binary classification loss for movement
#         Args:
#             alpha: Weight balancing factor between the two losses
#         """
#         self.alpha = alpha  # Balance between MSE loss and classification loss
#         self.bce_loss = nn.BCELoss()
#         self.mse_loss = nn.MSELoss()  # Initialize MSE loss
        
#     def __call__(self, return_preds, return_targets, move_preds, move_targets):
#         """
#         Calculate combined multi-task loss
#         Args:
#             return_preds: Predicted return ratios
#             return_targets: Actual return ratios
#             move_preds: Predicted movement probabilities
#             move_targets: Actual movement labels (0/1)
#         """
#         # Calculate MSE loss for return ratio prediction
#         return_loss = self.mse_loss(return_preds, return_targets)
        
#         # Calculate BCE loss for movement prediction
#         movement_loss = self.bce_loss(move_preds, move_targets)
        
#         # Combine the losses using alpha as weight
#         combined_loss = self.alpha * return_loss + (1 - self.alpha) * movement_loss
        
#         return combined_loss, return_loss, movement_loss

In [31]:
def prepare_weekly_data(week_idx, weekly_long_term_embeddings, weekly_inter_sector_embeddings, stock_sector_map):
    """
    Prepares training data for a specific week by aligning embeddings and labels
    Args:
        week_idx: The week index to prepare data for
        weekly_long_term_embeddings: Dictionary of long-term embeddings (by week, company)
        weekly_inter_sector_embeddings: Dictionary of sector embeddings (by week, sector)
        stock_sector_map: Mapping from stocks to their sectors
    Returns:
        Dictionary of aligned data ready for training
    """
    # Skip if this week doesn't have both types of embeddings
    if (week_idx not in weekly_long_term_embeddings or 
        week_idx not in weekly_inter_sector_embeddings):
        print(f'skipping week:{week_idx}')
        return None
    
    training_data = {
        'attentive_embs': [],
        'graph_embs': [],
        'sector_embs': [],
        'companies': [],
        'returns': [],
        'movements': []
    }
    
    # For each company that has data in this week
    for company_idx, company_data in weekly_long_term_embeddings[week_idx].items():
        # Get the stock's sector (assuming each stock belongs to only one sector)
        if company_idx not in stock_sector_map or len(stock_sector_map[company_idx]) == 0:
            continue
            
        sector_id = list(stock_sector_map[company_idx])[0]  # Take the first sector if multiple
        
        # Check if we have the sector embedding for this week
        if sector_id not in weekly_inter_sector_embeddings[week_idx]:
            continue
            
        # Extract embeddings
        attentive_emb = torch.tensor(company_data['attentive'], dtype=torch.float32)
        graph_emb = torch.tensor(company_data['graph'], dtype=torch.float32)
        sector_emb = weekly_inter_sector_embeddings[week_idx][sector_id]
        
        # Get return and movement labels for this company at this week
        if company_idx not in stock_returns_map:
            continue
            
        # Align with the correct week's return data
        # This is a simplification - you'll need to properly align with your data
        target_week = week_idx  # Use current week's return (or week_idx + 1 for next week)
        
        if 0 <= target_week < len(stock_returns_map[company_idx]):
            return_ratio = stock_returns_map[company_idx][target_week]
            movement = 1.0 if return_ratio > 0 else 0.0
            
            
            # Add to training data
            training_data['attentive_embs'].append(attentive_emb)
            training_data['graph_embs'].append(graph_emb)
            training_data['sector_embs'].append(sector_emb)
            training_data['companies'].append(company_idx)
            training_data['returns'].append(return_ratio)
            training_data['movements'].append(movement)
    
    # Convert to tensors if we have data
    if training_data['returns']:
        training_data['attentive_embs'] = torch.stack(training_data['attentive_embs'])
        training_data['graph_embs'] = torch.stack(training_data['graph_embs'])
        training_data['sector_embs'] = torch.stack(training_data['sector_embs'])
        training_data['returns'] = torch.tensor(training_data['returns'], dtype=torch.float32)
        training_data['movements'] = torch.tensor(training_data['movements'], dtype=torch.float32)
        return training_data
    else:
        return None

In [32]:
def calculate_rank_based_r2(predictions, targets):
    """
    Calculate R² score based on rankings rather than absolute values.
    
    Args:
        predictions: Predicted return ratios
        targets: Actual return ratios
        
    Returns:
        R² score based on ranking performance
    """
    # Convert tensors to numpy if needed
    if torch.is_tensor(predictions):
        predictions = predictions.detach().cpu().numpy()
    if torch.is_tensor(targets):
        targets = targets.detach().cpu().numpy()
    
    # Convert to rankings (scipy.stats.rankdata sorts in ascending order)
    from scipy import stats
    pred_ranks = stats.rankdata(-predictions)  # Negative to rank in descending order
    true_ranks = stats.rankdata(-targets)      # Higher returns should have lower rank numbers
    
    # Calculate R² using the ranking formula
    mean_true_rank = np.mean(true_ranks)
    ss_tot = np.sum((true_ranks - mean_true_rank) ** 2)
    ss_res = np.sum((true_ranks - pred_ranks) ** 2)
    
    if ss_tot == 0:  # Avoid division by zero
        return 0.0
    
    # R² formula: 1 - (residual sum of squares / total sum of squares)
    return 1 - (ss_res / ss_tot)

In [33]:
def train_fingat(epochs=50, learning_rate=0.001):
    """
    Training loop for the FinGAT model
    """
    # Import sklearn metrics for R2 calculation
    from sklearn.metrics import r2_score
    
    # Initialize model
    model = FinGAT(
        attentive_dim=HIDDEN_SIZE,
        graph_dim=HIDDEN_SIZE,
        sector_dim=HIDDEN_SIZE
    ).to(device)
    
    # Initialize optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = MultiTaskLoss(alpha=0.5)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        ranking_losses = 0.0
        movement_losses = 0.0
        batch_count = 0
        
        # For R2 calculation
        all_return_preds = []
        all_return_targets = []
        
        # Process each week's data
        for week_idx in sorted(weekly_long_term_embeddings.keys()):
            # Prepare data for this week
            week_data = prepare_weekly_data(
                week_idx, 
                weekly_long_term_embeddings, 
                weekly_inter_sector_embeddings,
                stock_sector_map
            )
            
            if week_data and len(week_data['returns']) > 1:  # Need at least 2 samples for ranking loss
                # Move to device
                attentive_embs = week_data['attentive_embs'].to(device)
                graph_embs = week_data['graph_embs'].to(device)
                sector_embs = week_data['sector_embs'].to(device)
                return_targets = week_data['returns'].to(device)
                movement_targets = week_data['movements'].to(device)
                
                # Forward pass
                return_preds, movement_preds = model(attentive_embs, graph_embs, sector_embs)
                
                # Compute loss
                loss, ranking_loss, move_loss = criterion(
                    return_preds, return_targets, movement_preds, movement_targets
                )
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Record losses
                total_loss += loss.item()
                ranking_losses += ranking_loss.item()
                movement_losses += move_loss.item()
                batch_count += 1
                
                # Collect predictions and targets for R2 calculation
                all_return_preds.extend(return_preds.detach().cpu().numpy())
                all_return_targets.extend(return_targets.detach().cpu().numpy())
        
        # Report progress
        if batch_count > 0:
            # Calculate standard R² score
            r2 = r2_score(all_return_targets, all_return_preds) if len(all_return_preds) > 1 else 0.0
            
            # Calculate rank-based R² score
            rank_r2 = calculate_rank_based_r2(np.array(all_return_preds), np.array(all_return_targets))
            
            print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/batch_count:.4f} "
                  f"(Ranking: {ranking_losses/batch_count:.4f}, Movement: {movement_losses/batch_count:.4f}, "
                  f"Standard R²: {r2:.4f}, Rank R²: {rank_r2:.4f})")
        
    return model

In [34]:
def evaluate_fingat(model):
    """
    Evaluate the FinGAT model's performance with additional MRR metrics
    """
    model.eval()
    
    all_return_preds = []
    all_return_targets = []
    all_movement_preds = []
    all_movement_targets = []
    
    # Store predictions by week for MRR calculation
    weekly_predictions = defaultdict(lambda: {'preds': [], 'targets': [], 'companies': []})
    
    with torch.no_grad():
        # Use the last week for evaluation
        eval_weeks = sorted(weekly_long_term_embeddings.keys())[-3:]  # Last 3 weeks for example
        
        for week_idx in eval_weeks:
            week_data = prepare_weekly_data(
                week_idx, 
                weekly_long_term_embeddings, 
                weekly_inter_sector_embeddings,
                stock_sector_map
            )
            
            if week_data and len(week_data['returns']) > 0:
                # Move to device
                attentive_embs = week_data['attentive_embs'].to(device)
                graph_embs = week_data['graph_embs'].to(device)
                sector_embs = week_data['sector_embs'].to(device)
                
                # Get predictions
                return_preds, movement_preds = model(attentive_embs, graph_embs, sector_embs)
                
                # Collect results
                return_preds_np = return_preds.cpu().numpy()
                return_targets_np = week_data['returns'].numpy()
                
                all_return_preds.extend(return_preds_np)
                all_return_targets.extend(return_targets_np)
                all_movement_preds.extend(movement_preds.cpu().numpy())
                all_movement_targets.extend(week_data['movements'].numpy())
                
                # Store by week for MRR calculation
                weekly_predictions[week_idx]['preds'].extend(return_preds_np)
                weekly_predictions[week_idx]['targets'].extend(return_targets_np)
                weekly_predictions[week_idx]['companies'].extend(week_data['companies'])
    
    # Calculate traditional metrics
    from scipy.stats import spearmanr
    corr, _ = spearmanr(all_return_preds, all_return_targets)
    
    # For movement: Binary classification metrics
    threshold = 0.5
    binary_preds = [1 if p > threshold else 0 for p in all_movement_preds]
    
    # Accuracy
    accuracy = sum(p == t for p, t in zip(binary_preds, all_movement_targets)) / len(binary_preds) if binary_preds else 0
    
    # Calculate Mean Reciprocal Rank (MRR) for different top-N values
    mrr_values = []      # Original MRR (top 1)
    mrr_top5_values = []
    mrr_top10_values = []
    mrr_top20_values = []
    mrr_top30_values = []
    
    for week_idx, week_data in weekly_predictions.items():
        if not week_data['preds']:
            continue
            
        # Create a list of (predicted return, actual return, company) tuples
        pred_target_pairs = list(zip(week_data['preds'], week_data['targets'], week_data['companies']))
        
        # Sort by predicted returns (highest to lowest) - this is our prediction ranking
        pred_ranking = sorted(pred_target_pairs, key=lambda x: x[0], reverse=True)
        
        # Create company to predicted rank mapping
        company_to_pred_rank = {company: i+1 for i, (_, _, company) in enumerate(pred_ranking)}
        
        # Create a ranking of companies based on true returns (highest to lowest)
        true_ranking = sorted(
            [(target, company) for (_, target, company) in pred_target_pairs],
            key=lambda x: x[0],
            reverse=True
        )
        
        # Calculate MRR@1 (original)
        if true_ranking:
            _, best_company = true_ranking[0]
            if best_company in company_to_pred_rank:
                mrr_values.append(1.0 / company_to_pred_rank[best_company])
        
        # Calculate MRR@5
        top_k_values = [5, 10, 20, 30]
        for k in top_k_values:
            mrr_at_k = []
            # Take top-k companies from true ranking (or all if less than k)
            for j, (_, company) in enumerate(true_ranking[:k]):
                if company in company_to_pred_rank:
                    mrr_at_k.append(1.0 / company_to_pred_rank[company])
            
            # Add to appropriate list
            if k == 5:
                mrr_top5_values.extend(mrr_at_k)
            elif k == 10:
                mrr_top10_values.extend(mrr_at_k)
            elif k == 20:
                mrr_top20_values.extend(mrr_at_k)
            elif k == 30:
                mrr_top30_values.extend(mrr_at_k)
    
    # Calculate MRR as average of reciprocal ranks
    mrr = sum(mrr_values) / len(mrr_values) if mrr_values else 0
    mrr_top5 = sum(mrr_top5_values) / len(mrr_top5_values) if mrr_top5_values else 0
    mrr_top10 = sum(mrr_top10_values) / len(mrr_top10_values) if mrr_top10_values else 0
    mrr_top20 = sum(mrr_top20_values) / len(mrr_top20_values) if mrr_top20_values else 0
    mrr_top30 = sum(mrr_top30_values) / len(mrr_top30_values) if mrr_top30_values else 0
    print("\nTop 5 Predicted Companies (Validation):")
    for i, (pred_return, _, company_id) in enumerate(pred_ranking[:min(5, len(pred_ranking))], 1):
        # Find the actual return for this company
        actual_return = next((target for target, comp in true_ranking if comp == company_id), "N/A")
        company_name = company_id_to_name.get(company_id, f"Unknown-{company_id}")
        print(f"{i}. Company: {company_name} (ID: {company_id}), Predicted Return: {pred_return:.6f}, Actual Return: {actual_return:.6f}")

    print("\nTop 5 Companies by Actual Returns (Validation):")
    for i, (true_return, company_id) in enumerate(true_ranking[:min(5, len(true_ranking))], 1):
        # Find the predicted rank
        pred_rank = next((idx+1 for idx, (_, _, comp) in enumerate(pred_ranking) if comp == company_id), "N/A")
        company_name = company_id_to_name.get(company_id, f"Unknown-{company_id}")
        print(f"{i}. Company: {company_name} (ID: {company_id}), Actual Return: {true_return:.6f}, Predicted Rank: {pred_rank}")
    print(f"Evaluation metrics:")
    print(f"Return ranking correlation: {corr:.4f}")
    print(f"Movement prediction accuracy: {accuracy:.4f}")
    print(f"Mean Reciprocal Rank (MRR@1): {mrr:.4f}")
    print(f"Mean Reciprocal Rank (MRR@5): {mrr_top5:.4f}")
    print(f"Mean Reciprocal Rank (MRR@10): {mrr_top10:.4f}")
    print(f"Mean Reciprocal Rank (MRR@20): {mrr_top20:.4f}")
    print(f"Mean Reciprocal Rank (MRR@30): {mrr_top30:.4f}")
    
    return corr, accuracy, mrr, mrr_top5, mrr_top10, mrr_top20, mrr_top30

In [35]:
# Train the model
fingat_model = train_fingat(epochs=10)

with torch.no_grad():
    # Save the model
    torch.save(fingat_model.state_dict(), "fingat_model.pth")
    print("Model saved as fingat_model.pth")
# Evaluate with the new MRR metric
evaluate_fingat(fingat_model)



/var/folders/gn/mtbdxhjn1697bp_qhyd6y0cr0000gn/T/ipykernel_75395/2324452217.py:63: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Scalar.cpp:23.)
  total_loss += loss.item()


Epoch 1/10 - Loss: 0.3971 (Ranking: 0.0999, Movement: 0.6942, Standard R²: -2.2781, Rank R²: -0.9783)
Epoch 2/10 - Loss: 0.3969 (Ranking: 0.0999, Movement: 0.6940, Standard R²: -2.6630, Rank R²: -1.0496)
Epoch 3/10 - Loss: 0.3967 (Ranking: 0.0999, Movement: 0.6936, Standard R²: -2.4091, Rank R²: -1.0469)
Epoch 4/10 - Loss: 0.3967 (Ranking: 0.0998, Movement: 0.6937, Standard R²: -4.7326, Rank R²: -1.0352)
Epoch 5/10 - Loss: 0.3967 (Ranking: 0.0998, Movement: 0.6936, Standard R²: -8.6121, Rank R²: -1.0193)
Epoch 6/10 - Loss: 0.3966 (Ranking: 0.0997, Movement: 0.6935, Standard R²: -13.9709, Rank R²: -1.0152)
Epoch 7/10 - Loss: 0.3966 (Ranking: 0.0998, Movement: 0.6934, Standard R²: -20.1163, Rank R²: -1.0373)
Epoch 8/10 - Loss: 0.3965 (Ranking: 0.0997, Movement: 0.6934, Standard R²: -20.0334, Rank R²: -1.0480)
Epoch 9/10 - Loss: 0.3965 (Ranking: 0.0997, Movement: 0.6933, Standard R²: -22.3288, Rank R²: -0.9930)
Epoch 10/10 - Loss: 0.3965 (Ranking: 0.0997, Movement: 0.6933, Standard R²: -2

(0.01546215480584007,
 0.54421768707483,
 0.024311118907926112,
 0.01077848561749482,
 0.011817830970975947,
 0.011235034129548568,
 0.010463184201282694)

In [38]:
# Create the validation dataset
val_dataset = HierarchicalStockDataset(df_val_date)
print(f"Validation dataset contains {val_dataset.__len__()} samples")

# Process validation data similarly to training data
val_company_sequences = defaultdict(list)

# Organize validation samples by company
for idx in range(len(val_dataset)):
    features, industry_id, company_id, return_ratio, movements = val_dataset[idx]
    company_id = company_id.item()
    industry_id = industry_id.item()
    
    val_company_sequences[company_id].append({
        'features': features,
        'industry_id': industry_id,
        'return_ratio': return_ratio.item(),
        'movements': movements.item(),
        'idx': idx
    })


# Process validation embeddings
val_attentive_embeddings = {}
val_stock_returns_map = defaultdict(list)
val_stock_movements_map = defaultdict(list)

# Create attentive embeddings for validation data
stsl_model = TransformerSequentialLearner(
    input_size=train_dataset[0][0].shape[1],  # num_features 
    hidden_size=HIDDEN_SIZE
).to(DEVICE)
stsl_model.eval()

with torch.no_grad():
    for company_id, sequences in val_company_sequences.items():
        # Sort by original index to maintain temporal ordering
        sequences.sort(key=lambda x: x['idx'])
        
        # Process all sequences for this company in order
        company_attentive_embeddings = []
        
        for seq in sequences:
            features = seq['features'].unsqueeze(0).to(DEVICE)
            industry_id = seq['industry_id']
            return_ratio = seq['return_ratio']
            movements = seq['movements']
            
            # Store return and movement information
            val_stock_returns_map[company_id].append(return_ratio)
            val_stock_movements_map[company_id].append(movements)
            
            # Get embeddings using the sequential model
            context, _ = stsl_model(features)
            company_attentive_embeddings.append(context.squeeze(0).cpu().numpy())
        
        # Store the entire temporal sequence for this company
        if company_attentive_embeddings:  # Only store if we have data
            val_attentive_embeddings[company_id] = np.stack(company_attentive_embeddings, axis=0)
print(f"Validation attentive embeddings processed: {len(val_attentive_embeddings)} companies")

# Function to evaluate on validation data
def evaluate_on_validation(trained_model):
    """
    Evaluate the FinGAT model on validation data by creating new embeddings from validation data
    """
    print("Creating validation embeddings from scratch...")
    
    # We already have val_attentive_embeddings from the previous code block
    # Now we need to create intra-sector graphs, process them through GAT, etc.
    
    # STEP 1: Create intra-sector graphs from validation data
    val_weekly_intra_sector_graphs = defaultdict(lambda: defaultdict(list))
    
    # Find maximum weeks in validation data
    max_val_weeks = max([len(emb) for emb in val_attentive_embeddings.values()]) if val_attentive_embeddings else 0
    
    # Process each timepoint (week) separately
    for week_idx in range(max_val_weeks):
        print(f"Processing validation week {week_idx}")
        
        # Group by sector
        for sector_id, sector_stocks in sector_stock_map.items():
            sector_features = []
            valid_indices = []
            
            # Collect stock embeddings in this sector-week
            for stock_idx in sector_stocks:
                if stock_idx in val_attentive_embeddings and week_idx < len(val_attentive_embeddings[stock_idx]):
                    tensor_embedding = torch.tensor(val_attentive_embeddings[stock_idx][week_idx], dtype=torch.float32)
                    sector_features.append(tensor_embedding)
                    valid_indices.append(stock_idx)
            
            # Create graph if >= 2 stocks
            if len(sector_features) >= 2:
                edge_index = []
                num_nodes = len(valid_indices)
                
                # Fully-connected edges without self-loops
                for i in range(num_nodes):
                    for j in range(num_nodes):
                        if i != j:
                            edge_index.append([i, j])
                
                # Store the graph with additional sector_id attribute
                val_weekly_intra_sector_graphs[week_idx][sector_id] = Data(
                    x=torch.stack(sector_features),
                    edge_index=torch.tensor(edge_index).t().contiguous(),
                    original_indices=valid_indices,
                    sector_id=sector_id
                )
    
    # STEP 2: Process validation graphs through IntraSectorGAT
    val_sector_embeddings = []
    gat_model = IntraSectorGAT().to(device)  # Reuse the model architecture
    gat_model.eval()
    
    with torch.no_grad():
        for week_idx in val_weekly_intra_sector_graphs:
            for sector_id, graph in val_weekly_intra_sector_graphs[week_idx].items():
                graph = graph.to(device)
                out = gat_model(graph)
                val_sector_embeddings.append({
                    'embeddings': out.cpu(),
                    'original_indices': graph.original_indices,
                    'sector_id': graph.sector_id,
                    'week_idx': week_idx
                })
    
    # STEP 3: Create pooled embeddings for sectors
    val_weekly_sector_pooled_embeddings = defaultdict(dict)
    
    if val_sector_embeddings:
        embedding_dim = val_sector_embeddings[0]['embeddings'].shape[1]
        
        for week_idx in sorted(val_weekly_intra_sector_graphs.keys()):
            for sector_id in val_weekly_intra_sector_graphs[week_idx]:
                graph = val_weekly_intra_sector_graphs[week_idx][sector_id]
                pooled_embedding = torch.max(graph.x, dim=0)[0]  # Max pooling
                val_weekly_sector_pooled_embeddings[week_idx][sector_id] = pooled_embedding
    
    # STEP 4: Process inter-sector relationships
    val_weekly_inter_sector_embeddings = defaultdict(dict)
    
    if val_sector_embeddings:
        inter_sector_model = InterSectorGAT(
            in_channels=embedding_dim,
            hidden_channels=HIDDEN_SIZE
        ).to(device)
        inter_sector_model.eval()
        
        with torch.no_grad():
            for week_idx in sorted(val_weekly_sector_pooled_embeddings.keys()):
                sectors_this_week = val_weekly_sector_pooled_embeddings[week_idx]
                
                # Skip weeks with less than 2 sectors
                num_sectors = len(sectors_this_week)
                if num_sectors < 2:
                    continue
                    
                # Create mapping from sector IDs to indices
                sector_ids = list(sectors_this_week.keys())
                sector_id_to_idx = {sector_id: idx for idx, sector_id in enumerate(sector_ids)}
                
                # Create features tensor
                sector_features = torch.zeros((num_sectors, embedding_dim), dtype=torch.float32)
                
                # Populate features tensor
                for sector_id, idx in sector_id_to_idx.items():
                    sector_features[idx] = sectors_this_week[sector_id]
                
                # Create fully connected graph without self-loops
                edge_indices = []
                for i in range(num_sectors):
                    for j in range(num_sectors):
                        if i != j:  # No self-loops
                            edge_indices.append([i, j])
                
                edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
                
                # Process through GAT model
                sector_features = sector_features.to(device)
                edge_index = edge_index.to(device)
                inter_sector_embeddings = inter_sector_model(sector_features, edge_index)
                
                # Store by original sector ID
                inter_sector_embeddings = inter_sector_embeddings.cpu()
                for sector_id, idx in sector_id_to_idx.items():
                    val_weekly_inter_sector_embeddings[week_idx][sector_id] = inter_sector_embeddings[idx]
    
    # STEP 5: Organize company sector embeddings
    val_company_sector_embeddings = defaultdict(lambda: defaultdict(list))
    
    for sector_data in val_sector_embeddings:
        week_idx = sector_data['week_idx']
        original_indices = sector_data['original_indices']
        embeddings = sector_data['embeddings']
        
        for i, company_idx in enumerate(original_indices):
            val_company_sector_embeddings[company_idx][week_idx] = embeddings[i]
    
    # STEP 6: Process long-term embeddings
    val_weekly_long_term_embeddings = defaultdict(lambda: defaultdict(dict))
    SEQ_LENGTH = 4  # Same as training
    
    if val_sector_embeddings:
        # Initialize long term sequential learners
        long_term_g = LongTermTransformerLearner(
            input_size=val_sector_embeddings[0]['embeddings'].shape[1],
            hidden_size=HIDDEN_SIZE,
            lookback_weeks=SEQ_LENGTH
        ).to(device)
        
        long_term_a = LongTermTransformerLearner(
            input_size=next(iter(val_attentive_embeddings.values())).shape[1],
            hidden_size=HIDDEN_SIZE,
            lookback_weeks=SEQ_LENGTH
        ).to(device)
        
        with torch.no_grad():
            # Get the maximum week index
            if val_company_sector_embeddings:
                max_weeks = max([max(weekly_data.keys()) for weekly_data in val_company_sector_embeddings.values()])
                
                # Process each week from SEQ_LENGTH onwards
                for current_week in range(SEQ_LENGTH, max_weeks + 1):
                    print(f"Processing validation long-term embeddings for week {current_week}")
                    # Process each company for this week
                    for company_idx, weekly_data in val_company_sector_embeddings.items():
                        # Check if we have data for this company in this week
                        if current_week not in weekly_data:
                            continue
                            
                        # Check if we have enough history
                        history_weeks = [w for w in range(current_week - SEQ_LENGTH, current_week)]
                        if not all(w in weekly_data for w in history_weeks):
                            continue
                            
                        # Gather the sliding window of embeddings
                        graph_window = []
                        for w in history_weeks:
                            if w in weekly_data:
                                graph_emb = torch.tensor(weekly_data[w]).unsqueeze(0).to(device)
                                graph_window.append(graph_emb)
                        
                        # Process attentive embeddings if available
                        if company_idx in val_attentive_embeddings:
                            attentive_seqs = val_attentive_embeddings[company_idx]
                            
                            if len(attentive_seqs) >= SEQ_LENGTH:
                                attentive_window = []
                                for i in range(SEQ_LENGTH):
                                    seq_idx = len(attentive_seqs) - SEQ_LENGTH + i
                                    if seq_idx >= 0 and seq_idx < len(attentive_seqs):
                                        att_emb = torch.tensor(attentive_seqs[seq_idx], dtype=torch.float32).unsqueeze(0).to(device)
                                        attentive_window.append(att_emb)
                                
                                if len(graph_window) == SEQ_LENGTH and len(attentive_window) == SEQ_LENGTH:
                                    tau_G = long_term_g(graph_window)
                                    tau_A = long_term_a(attentive_window)
                                    
                                    # Store the results
                                    val_weekly_long_term_embeddings[current_week][company_idx] = {
                                        'graph': tau_G.cpu().numpy(),
                                        'attentive': tau_A.cpu().numpy()
                                    }
    
    # STEP 7: Create final evaluation data using the latest week
    val_data = {
        'attentive_embs': [],
        'graph_embs': [],
        'sector_embs': [],
        'returns': [],
        'movements': [],
        'companies': []
    }
    
    # Use the last week for prediction
    if val_weekly_long_term_embeddings:
        last_week = max(val_weekly_long_term_embeddings.keys())
        print(f"Using week {last_week} for final evaluation")
        
        for company_idx, company_data in val_weekly_long_term_embeddings[last_week].items():
            # Get the stock's sector
            if company_idx not in stock_sector_map or len(stock_sector_map[company_idx]) == 0:
                continue
                
            sector_id = list(stock_sector_map[company_idx])[0]
            
            # Check if we have the sector embedding for this week
            if sector_id not in val_weekly_inter_sector_embeddings[last_week]:
                continue
                
            # Extract embeddings
            attentive_emb = torch.tensor(company_data['attentive'], dtype=torch.float32)
            graph_emb = torch.tensor(company_data['graph'], dtype=torch.float32)
            sector_emb = val_weekly_inter_sector_embeddings[last_week][sector_id]
            
            # Get return and movement labels
            if company_idx in val_stock_returns_map and val_stock_returns_map[company_idx]:
                return_ratio = val_stock_returns_map[company_idx][0]
                movement = 1.0 if return_ratio > 0 else 0.0
                
                # Add to evaluation data
                val_data['attentive_embs'].append(attentive_emb)
                val_data['graph_embs'].append(graph_emb)
                val_data['sector_embs'].append(sector_emb)
                val_data['companies'].append(company_idx)
                val_data['returns'].append(return_ratio)
                val_data['movements'].append(movement)
    
    # Perform evaluation if we have data
    if val_data['returns']:
        val_data['attentive_embs'] = torch.stack(val_data['attentive_embs'])
        val_data['graph_embs'] = torch.stack(val_data['graph_embs'])
        val_data['sector_embs'] = torch.stack(val_data['sector_embs'])
        val_data['returns'] = torch.tensor(val_data['returns'], dtype=torch.float32)
        val_data['movements'] = torch.tensor(val_data['movements'], dtype=torch.float32)
        
        # Evaluate
        trained_model.eval()
        with torch.no_grad():
            # Move to device
            attentive_embs = val_data['attentive_embs'].to(device)
            graph_embs = val_data['graph_embs'].to(device)
            sector_embs = val_data['sector_embs'].to(device)
            
            # Get predictions
            return_preds, movement_preds = trained_model(attentive_embs, graph_embs, sector_embs)
            
            # Calculate metrics (same as before)
            return_preds_np = return_preds.cpu().numpy()
            return_targets_np = val_data['returns'].numpy()
            movement_preds_np = movement_preds.cpu().numpy()
            movement_targets_np = val_data['movements'].numpy()
            
            # Correlation for returns
            from scipy.stats import spearmanr
            corr, _ = spearmanr(return_preds_np, return_targets_np)
            
            # Accuracy for movement prediction
            threshold = 0.5
            binary_preds = [1 if p > threshold else 0 for p in movement_preds_np]
            accuracy = sum(p == t for p, t in zip(binary_preds, movement_targets_np)) / len(binary_preds)
            
            # R² for returns
            from sklearn.metrics import r2_score
            r2 = r2_score(return_targets_np, return_preds_np)
            
            # Calculate MRR metrics
            pred_target_pairs = list(zip(return_preds_np, return_targets_np, val_data['companies']))
            pred_ranking = sorted(pred_target_pairs, key=lambda x: x[0], reverse=True)
            company_to_pred_rank = {company: i+1 for i, (_, _, company) in enumerate(pred_ranking)}
            
            true_ranking = sorted(
                [(target, company) for (_, target, company) in pred_target_pairs],
                key=lambda x: x[0], 
                reverse=True
            )
            
            mrr_values = {}
            top_k_values = [5, 10, 20, 30, 100]
            
            for k in top_k_values:
                mrr_at_k = []
                for j, (_, company) in enumerate(true_ranking[:min(k, len(true_ranking))]):
                    if company in company_to_pred_rank:
                        mrr_at_k.append(1.0 / company_to_pred_rank[company])
                
                mrr_values[k] = sum(mrr_at_k) / len(mrr_at_k) if mrr_at_k else 0
            
            # Print results
            print(f"\nValidation Metrics (using fresh embeddings):")
            print(f"Number of validation samples: {len(return_preds_np)}")
            print(f"Return prediction correlation: {corr:.4f}")
            print(f"Movement prediction accuracy: {accuracy:.4f}")
            print(f"R² score: {r2:.4f}")
            
            print("\nMean Reciprocal Rank (MRR) Metrics:")
            for k in top_k_values:
                print(f"MRR@{k}: {mrr_values[k]:.4f}")
            
            return corr, accuracy, r2, mrr_values
    else:
        print("No validation data available for evaluation")
        return None, None, None, None
    
# After training the model, evaluate on validation data with freshly created embeddings
print("\n--- Evaluating on Validation Data with Fresh Embeddings ---")
val_metrics = evaluate_on_validation(fingat_model)


Validation dataset contains 42336 samples
Validation attentive embeddings processed: 441 companies

--- Evaluating on Validation Data with Fresh Embeddings ---
Creating validation embeddings from scratch...
Processing validation week 0
Processing validation week 1
Processing validation week 2
Processing validation week 3
Processing validation week 4
Processing validation week 5
Processing validation week 6
Processing validation week 7
Processing validation week 8
Processing validation week 9
Processing validation week 10
Processing validation week 11
Processing validation week 12
Processing validation week 13
Processing validation week 14
Processing validation week 15
Processing validation week 16
Processing validation week 17
Processing validation week 18
Processing validation week 19
Processing validation week 20
Processing validation week 21
Processing validation week 22
Processing validation week 23
Processing validation week 24
Processing validation week 25
Processing validation w

/var/folders/gn/mtbdxhjn1697bp_qhyd6y0cr0000gn/T/ipykernel_75395/3810979502.py:243: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  graph_emb = torch.tensor(weekly_data[w]).unsqueeze(0).to(device)


Processing validation long-term embeddings for week 5
Processing validation long-term embeddings for week 6
Processing validation long-term embeddings for week 7
Processing validation long-term embeddings for week 8
Processing validation long-term embeddings for week 9
Processing validation long-term embeddings for week 10
Processing validation long-term embeddings for week 11
Processing validation long-term embeddings for week 12
Processing validation long-term embeddings for week 13
Processing validation long-term embeddings for week 14
Processing validation long-term embeddings for week 15
Processing validation long-term embeddings for week 16
Processing validation long-term embeddings for week 17
Processing validation long-term embeddings for week 18
Processing validation long-term embeddings for week 19
Processing validation long-term embeddings for week 20
Processing validation long-term embeddings for week 21
Processing validation long-term embeddings for week 22
Processing vali